In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
import os

# Hyperparameters
batch_size = 128
z_dim = 100  # Size of the noise vector
image_size = 28
channels = 1
epochs = 50
lr = 0.0002
beta1 = 0.5  # Adam optimizer beta1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create output folder
os.makedirs("generated_images", exist_ok=True)

In [ ]:
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize between [-1, 1]
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=True, download=True, transform=transform),
    batch_size=batch_size,
    shuffle=True
)

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 494kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.50MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.97MB/s]


In [ ]:
class Generator(nn.Module):
    def __init__(self, z_dim):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # Input: (N, z_dim, 1, 1)
            nn.ConvTranspose2d(z_dim, 256, 7, 1, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 1, 4, 2, 1, bias=False),
            nn.Tanh()  # Output range [-1, 1]
        )

    def forward(self, z):
        return self.net(z)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            # Input: (N, 1, 28, 28)
            nn.Conv2d(1, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        return self.net(img)

In [ ]:
generator = Generator(z_dim).to(device)
discriminator = Discriminator().to(device)

# Binary Cross Entropy loss
criterion = nn.BCELoss()

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
def generate_and_save_images(epoch):
    generator.eval()
    with torch.no_grad():
        z = torch.randn(64, z_dim, 1, 1).to(device)
        fake_images = generator(z)
        fake_images = fake_images * 0.5 + 0.5  # Denormalize to [0,1]
        save_image(fake_images, f"generated_images/sample_epoch_{epoch}.png", nrow=8)
    generator.train()

In [ ]:
k = 3  # Generator updates per iteration
p = 1  # Discriminator updates per iteration

In [ ]:
for epoch in range(1, epochs + 1):
    for i, (real_imgs, _) in enumerate(train_loader):
        batch_size_curr = real_imgs.size(0)
        real_imgs = real_imgs.to(device)

        real = torch.ones(batch_size_curr, 1, device=device)
        fake = torch.zeros(batch_size_curr, 1, device=device)

        ### ---- Train Discriminator p times ---- ###
        for _ in range(p):
            z = torch.randn(batch_size_curr, z_dim, 1, 1, device=device)
            fake_imgs = generator(z)

            # Real
            real_validity = discriminator(real_imgs)
            d_real_loss = criterion(real_validity, real)

            # Fake
            fake_validity = discriminator(fake_imgs.detach())
            d_fake_loss = criterion(fake_validity, fake)

            d_loss = d_real_loss + d_fake_loss

            optimizer_D.zero_grad()
            d_loss.backward()
            optimizer_D.step()

        ### ---- Train Generator k times ---- ###
        for _ in range(k):
            z = torch.randn(batch_size_curr, z_dim, 1, 1, device=device)
            fake_imgs = generator(z)

            validity = discriminator(fake_imgs)
            g_loss = criterion(validity, real)  # fool D → label as real

            optimizer_G.zero_grad()
            g_loss.backward()
            optimizer_G.step()

        if i % 200 == 0:
            print(f"[Epoch {epoch}/{epochs}] [Batch {i}/{len(train_loader)}] "
                  f"D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")

    # Save sample images
    generator.eval()
    with torch.no_grad():
        z = torch.randn(64, z_dim, 1, 1, device=device)
        samples = generator(z)
        samples = samples * 0.5 + 0.5  # Denormalize
        save_image(samples, f"generated_images/epoch_{epoch}.png", nrow=8)
    generator.train()

[Epoch 1/50] [Batch 0/469] D Loss: 1.4766 | G Loss: 0.3533
[Epoch 1/50] [Batch 200/469] D Loss: 1.2782 | G Loss: 1.0192
[Epoch 1/50] [Batch 400/469] D Loss: 1.2132 | G Loss: 0.8656
[Epoch 2/50] [Batch 0/469] D Loss: 1.1569 | G Loss: 0.7870
[Epoch 2/50] [Batch 200/469] D Loss: 1.1923 | G Loss: 0.8132
[Epoch 2/50] [Batch 400/469] D Loss: 1.2368 | G Loss: 0.8995
[Epoch 3/50] [Batch 0/469] D Loss: 1.2196 | G Loss: 0.9452
[Epoch 3/50] [Batch 200/469] D Loss: 1.1424 | G Loss: 0.8786
[Epoch 3/50] [Batch 400/469] D Loss: 1.1655 | G Loss: 0.9409
[Epoch 4/50] [Batch 0/469] D Loss: 1.1530 | G Loss: 0.9051
[Epoch 4/50] [Batch 200/469] D Loss: 1.2202 | G Loss: 0.4291
[Epoch 4/50] [Batch 400/469] D Loss: 1.1605 | G Loss: 0.8143
[Epoch 5/50] [Batch 0/469] D Loss: 1.1694 | G Loss: 0.8676
[Epoch 5/50] [Batch 200/469] D Loss: 1.2300 | G Loss: 0.7307
[Epoch 5/50] [Batch 400/469] D Loss: 1.0831 | G Loss: 0.6974
[Epoch 6/50] [Batch 0/469] D Loss: 1.1577 | G Loss: 1.0701


KeyboardInterrupt: 

In [ ]:
import os
from torchvision.utils import save_image

def generate_images_dcgan_individually(generator, num_samples=512, save_dir="generated_images", prefix="img"):
    """
    Generates images using a DCGAN generator and saves each one individually.

    Args:
        generator (nn.Module): Trained DCGAN generator model
        num_samples (int): Number of images to generate
        save_dir (str): Directory to save generated images
        prefix (str): Filename prefix for saved images
    """
    generator.eval()
    z = torch.randn(num_samples, z_dim, 1, 1).to(device)

    # Ensure output directory exists
    os.makedirs(save_dir, exist_ok=True)

    with torch.no_grad():
        gen_imgs = generator(z)
        gen_imgs = gen_imgs * 0.5 + 0.5  # Denormalize from [-1, 1] to [0, 1]

        for i, img in enumerate(gen_imgs):
            save_path = os.path.join(save_dir, f"{prefix}_{i:04d}.png")
            save_image(img, save_path)

    print(f"Saved {num_samples} images to '{save_dir}'")
    return gen_imgs

In [ ]:
generate_images_dcgan_individually(generator, num_samples=1024)

Saved 1024 images to 'generated_images'


tensor([[[[1.5657e-03, 1.3442e-03, 5.8168e-04,  ..., 5.6773e-05,
           1.1784e-03, 3.1164e-03],
          [1.2910e-03, 2.3162e-03, 4.2453e-04,  ..., 1.1325e-06,
           4.9919e-05, 1.9804e-03],
          [3.2893e-04, 8.6039e-05, 7.8291e-05,  ..., 8.9407e-08,
           5.0545e-05, 1.3902e-03],
          ...,
          [1.3358e-03, 1.8689e-04, 1.0073e-05,  ..., 0.0000e+00,
           1.4305e-06, 7.2151e-05],
          [5.4428e-04, 6.7204e-05, 8.8215e-06,  ..., 3.2783e-07,
           1.5378e-05, 5.8949e-05],
          [7.9972e-04, 1.0911e-04, 1.6123e-05,  ..., 4.8071e-05,
           1.1522e-04, 4.2784e-04]]],


        [[[1.8896e-03, 6.4728e-04, 3.4165e-04,  ..., 3.0994e-06,
           1.1185e-04, 9.5683e-04],
          [7.4792e-04, 1.4174e-04, 6.0588e-05,  ..., 2.3842e-07,
           7.0035e-06, 1.1083e-04],
          [1.5318e-04, 1.5110e-05, 3.0220e-05,  ..., 0.0000e+00,
           1.4603e-06, 3.0845e-05],
          ...,
          [1.4430e-04, 4.9174e-06, 2.4140e-06,  ..., 8.94

In [ ]:
import os
from torchvision.utils import save_image
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def save_random_mnist_images(save_dir="real_images", num_samples=1024, prefix="mnist", batch_size=128):
    """
    Extracts `num_samples` random MNIST images (from any class) and saves them individually.

    Args:
        save_dir (str): Folder to save the images
        num_samples (int): Total number of images to save
        prefix (str): Prefix for image filenames
        batch_size (int): Batch size used for loading MNIST
    """
    os.makedirs(save_dir, exist_ok=True)

    # MNIST normalization to [-1, 1] for FID
    transform = transforms.Compose([
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    saved = 0
    for batch_imgs, _ in loader:
        for img in batch_imgs:
            if saved >= num_samples:
                print(f"✅ Saved {num_samples} MNIST images to '{save_dir}'")
                return
            img_path = os.path.join(save_dir, f"{prefix}_{saved:04d}.png")
            save_image(img, img_path)
            saved += 1

# Example usage:
save_random_mnist_images()

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 495kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.52MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.94MB/s]


✅ Saved 1024 MNIST images to 'real_images'


In [ ]:
# FID Score Computation from Two Image Folders
# ---------------------------------------------
# This version computes FID by comparing real and generated images stored in folders
# Requirements: pip install torchvision scipy tqdm

import os
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import numpy as np
from scipy import linalg
from tqdm import tqdm

from torchvision.models import inception_v3, Inception_V3_Weights

from torchvision.models import inception_v3, Inception_V3_Weights
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import inception_v3, Inception_V3_Weights
import torch.nn as nn
import torch.nn.functional as F
import torch

class InceptionV3FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        # Load pretrained Inception with required weights
        weights = Inception_V3_Weights.IMAGENET1K_V1
        self.model = inception_v3(weights=weights, aux_logits=True)
        self.model.eval()
        for param in self.model.parameters():
            param.requires_grad = False

    def forward(self, x):
        with torch.no_grad():
            if x.shape[2] != 299 or x.shape[3] != 299:
                x = F.interpolate(x, size=(299, 299), mode='bilinear', align_corners=False)
            # Directly extract features before the final FC layer
            x = self.model.Conv2d_1a_3x3(x)
            x = self.model.Conv2d_2a_3x3(x)
            x = self.model.Conv2d_2b_3x3(x)
            x = self.model.maxpool1(x)
            x = self.model.Conv2d_3b_1x1(x)
            x = self.model.Conv2d_4a_3x3(x)
            x = self.model.maxpool2(x)
            x = self.model.Mixed_5b(x)
            x = self.model.Mixed_5c(x)
            x = self.model.Mixed_5d(x)
            x = self.model.Mixed_6a(x)
            x = self.model.Mixed_6b(x)
            x = self.model.Mixed_6c(x)
            x = self.model.Mixed_6d(x)
            x = self.model.Mixed_6e(x)
            x = self.model.Mixed_7a(x)
            x = self.model.Mixed_7b(x)
            x = self.model.Mixed_7c(x)
            x = self.model.avgpool(x)  # Shape: [B, 2048, 1, 1]
            x = x.view(x.size(0), -1)  # Shape: [B, 2048]
            return x
# ---------------------------------------------
# Step 2: Load Images from Folder
# ---------------------------------------------
def get_image_loader(folder_path, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # Normalize to [-1, 1]
    ])
    dataset = datasets.ImageFolder(folder_path, transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    return loader

# ---------------------------------------------
# Step 3: Extract Features using InceptionV3
# ---------------------------------------------
def get_activations(dataloader, model, device):
    model.eval()
    activations = []
    for images, _ in tqdm(dataloader, desc="Extracting features"):
        images = images.to(device)
        feats = model(images)
        activations.append(feats.cpu().numpy())
    return np.concatenate(activations, axis=0)

# ---------------------------------------------
# Step 4: Compute Fréchet Distance (FID Formula)
# ---------------------------------------------
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    diff = mu1 - mu2
    covmean, _ = linalg.sqrtm(sigma1 @ sigma2, disp=False)  # Matrix square root of product of covariances
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = diff.dot(diff) + np.trace(sigma1 + sigma2 - 2 * covmean)
    return fid

# ---------------------------------------------
# Step 5: Main Function to Compute FID
# ---------------------------------------------
def compute_fid_from_folders(real_folder, gen_folder, device):
    # Load datasets
    real_loader = get_image_loader(real_folder)
    gen_loader = get_image_loader(gen_folder)

    # Load feature extractor
    feature_extractor = InceptionV3FeatureExtractor().to(device)

    # Get activations for both sets
    real_feats = get_activations(real_loader, feature_extractor, device)
    gen_feats = get_activations(gen_loader, feature_extractor, device)

    # Compute mean and covariance of activations
    mu1, sigma1 = real_feats.mean(axis=0), np.cov(real_feats, rowvar=False)
    mu2, sigma2 = gen_feats.mean(axis=0), np.cov(gen_feats, rowvar=False)

    # Compute the FID score
    fid = calculate_frechet_distance(mu1, sigma1, mu2, sigma2)
    return fid

# ---------------------------------------------
# Example usage (replace with your paths)
# ---------------------------------------------
if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    real_path = "real_images"      # Folder containing real images (must be in subfolder, e.g., real_images/class_x/)
    generated_path = "generated_images"  # Folder containing generated images (same structure)

    fid_score = compute_fid_from_folders(real_path, generated_path, device)
    print(f"FID Score: {fid_score:.4f}")

Extracting features: 100%|██████████| 32/32 [06:45<00:00, 12.69s/it]


FID Score: 39.2303


In [ ]:
current_dir = os.getcwd()
folders = [f for f in os.listdir(current_dir) if os.path.isdir(os.path.join(current_dir, f))]

print("📁 Folders in current directory:")
for folder in folders:
    print(f" - {folder}")

📁 Folders in current directory:
 - .config
 - real_images
 - data
 - MNIST
 - generated_images
 - sample_data


In [ ]:
import os
import shutil

def wrap_in_class_folder(base_dir, class_name="class_x"):
    class_dir = os.path.join(base_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

    for fname in os.listdir(base_dir):
        path = os.path.join(base_dir, fname)
        if os.path.isfile(path):
            shutil.move(path, os.path.join(class_dir, fname))

# Example usage
wrap_in_class_folder("real_images")
wrap_in_class_folder("generated_images")

In [ ]:
import shutil
import os

for folder in ["real_images", "generated_images"]:
    checkpoints = os.path.join(folder, ".ipynb_checkpoints")
    if os.path.exists(checkpoints):
        shutil.rmtree(checkpoints)
        print(f"Removed: {checkpoints}")

Removed: generated_images/.ipynb_checkpoints
